# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle
import os.path

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'data/train.p'
validation_file = 'data/valid.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results
import numpy as np

# Number of training examples
n_train = len(X_train)

# Number of validation examples
n_validation = len(X_valid)

# Number of testing examples.
n_test = len(X_test)

# What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import flatten


In [ ]:
import csv

# Read "signnames.csv" to "signnames" dictionary
def load_signnames():
    signnames = {}
    with open('signnames.csv') as csvFile:
        reader = csv.DictReader(csvFile)
        for row in reader:
            signnames[row['ClassId']] = row['SignName']
    return signnames

signnames = load_signnames()

In [ ]:
# Visualizations will be shown in the notebook.
%matplotlib inline

def show_random_image(X, y):
    index = np.random.randint(0, len(X))
    image = X_train[index].squeeze()
    plt.figure(figsize=(1,1))
    plt.imshow(image)
    print('Sign ({}): {}'.format(index, signnames[str(y_train[index])]))

def show_distribution(y_data, title, filename):
    label_distribution = np.unique(y_data, return_counts = True)
    fig = plt.figure()
    plt.title(title)
    plt.xlabel('Sign ID')
    plt.ylabel('Count')
    plt.xticks(np.arange(0, n_classes, 5))
    plt.bar(label_distribution[0], label_distribution[1])
    plt.savefig(filename)
    plt.show()

show_random_image(X_train, y_train)
show_distribution(y_train, 'Distribution of Signs - Training Set', 'output/training_data_distribution.png')
show_distribution(y_valid, 'Distribution of Signs - Validation Set', 'output/validation_data_distribution.png')
show_distribution(y_test, 'Distribution of Signs - Test Set', 'output/test_data_distribution.png')

In [ ]:
# Pick 'num_per_sign' for each sign type.
# Returns array of sets
def pick_signs(X, y, num_per_sign = 10):
    n_train = X.shape[0]
    n_classes = len(np.unique(y_train))
    picked_signs = [set() for i in range(n_classes)]
    completed = 0
    while completed < n_classes:
        index = np.random.randint(0, n_train)
        sign_id = y[index]
        if len(picked_signs[sign_id]) < num_per_sign:
            picked_signs[sign_id].add(index)
            if len(picked_signs[sign_id]) == num_per_sign:
                completed += 1
    return picked_signs

def pick_and_show_signs():
    num_per_sign = 10
    picked_signs = pick_signs(X_train, y_train, num_per_sign)
    print('Picked', num_per_sign * n_classes, 'signs')

    print('Plotting ', end = '', flush = True)
    plt.figure(figsize=(num_per_sign, n_classes))
    row = col = 1
    for sign_set in picked_signs:
        for sign in sign_set:
            plt.subplot(n_classes, num_per_sign, (row - 1) * num_per_sign + col).axis('off')
            plt.imshow(X_train[sign])
            col += 1
            print('.', end = '', flush = True)
        row += 1
        col = 1
        if row % 5 == 0:
            print()
    print()
    plt.savefig('output/random_picked_signs.png', bbox_inches='tight')
    print('Picked signs saved')
    
pick_and_show_signs()

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
from skimage import transform
from skimage import img_as_ubyte
import time
import warnings

train_distribution = np.unique(y_train, return_counts = True)
train_sign_counts = train_distribution[1]
average_count_per_sign = np.mean(train_sign_counts)

# skimage.transform.rotate and rescale will change image.dtype to float64
# This is to convert image back to uint8
def convert_image_dtype_to_uint8(image):
    converted_image = None
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        converted_image = img_as_ubyte(image)
    return converted_image

def flip_image(image, label):
    flipped_images = []
    flipped_labels = []
    # Allow flip left-right and keep the label
    fliplr_allowed = {9, 11, 12, 13, 14, 15, 17, 18, 22, 26, 27, 30, 32, 35, 40}
    # Allow flip up-down and keep the label
    flipud_allowed = {1, 5, 12, 15, 17, 32, 40}
    # Allow flip left-right and change the label
    cross_flippable = {19:20, 20:19, 33:34, 34:33, 36:37, 37:36, 38:39, 39:38}
    if label in flipud_allowed:
        flipped_images.append(np.flipud(image))
        flipped_labels.append(label)
    if label in fliplr_allowed:
        flipped_images.append(np.fliplr(image))
        flipped_labels.append(label)
    if label in cross_flippable:
        flipped_images.append(np.fliplr(image))
        flipped_labels.append(cross_flippable[label])
    return (flipped_images, flipped_labels)

DEFAULT_ROTATE_ANGLE_RANGE = 25
def rotate_image(image, count):
    rotated_images = []
    angle_range = DEFAULT_ROTATE_ANGLE_RANGE
    if count > 0:
        angles = np.random.uniform(-angle_range, angle_range, count)
        for angle in angles:
            rotated_image = transform.rotate(image, angle, mode='edge')
            rotated_images.append(convert_image_dtype_to_uint8(rotated_image))        
    return rotated_images

def get_augment_number(label):
    number = int(average_count_per_sign / train_sign_counts[label]) * 2 + 1
    return number

DEFAULT_MAX_SCALE_RATE = 1.2
def scale_image(image, count):
    scaled_images = []
    if count > 0:
        scale_factors = np.random.uniform(1.0, DEFAULT_MAX_SCALE_RATE, count)
        for factor in scale_factors:
            scaled_image = transform.rescale(image, (factor, factor), mode='edge')
            scaled_image = transform.resize(scaled_image, image.shape, mode='edge')
            scaled_images.append(convert_image_dtype_to_uint8(scaled_image))
    return scaled_images
    
def augment_image(image, label):
    augmented_images = []
    augmented_labels = []
    # Flip image
    flipped_images, flipped_labels = flip_image(image, label)
    if len(flipped_images) > 0:
        augmented_images.extend(flipped_images)
        augmented_labels.extend(flipped_labels)
    
    # Rotate image
    count = get_augment_number(label)
    rotated_images = rotate_image(image, count)
    if len(rotated_images) > 0:
        augmented_images.extend(rotated_images)
        augmented_labels.extend(np.full(len(rotated_images), label))

    scaled_images = scale_image(image, count)
    if len(scaled_images) > 0:
        augmented_images.extend(scaled_images)
        augmented_labels.extend(np.full(len(scaled_images), label))

    return (augmented_images, augmented_labels)

def augment_images(X_data, y_data):
    X_augmented = []
    y_augmented = []
    count = 0
    start_time = time.time()
    for image, label in zip(X_data, y_data):
        count += 1
        new_images, new_labels = augment_image(image, label)
        X_augmented.extend(new_images)
        y_augmented.extend(new_labels)
        if count % 2000 == 0:
            print('Completed {} images. # of new images: {}, time used: {:.3f}'.format(count, len(X_augmented), time.time() - start_time))

    print('Completed {} images. # of new images: {}, time used: {:.3f}'.format(count, len(X_augmented), time.time() - start_time))
    return (np.stack(X_augmented), np.stack(y_augmented))

def save_and_show_augmented_image():
    index = np.random.randint(0, n_train)
    print('index = {}, sign id: {}'.format(index, y_train[index]))
    aug_images, aug_labels = augment_image(X_train[index], y_train[index])
    plt.figure()
    plt.imshow(X_train[index])
    for aug_image, aug_label in zip(aug_images, aug_labels):
        plt.figure()
        plt.imshow(aug_image)
        print('aug_label: {}, aug_image.dtype: {}'.format(aug_label, aug_image.dtype))

def run_augmentation(X, y):
    print('Augmenting images ...')
    X_extended, y_extended = augment_images(X, y)
    print('Augmented images. X_extended.shape: {}, y_extended.shape: {}'.format(X_extended.shape, y_extended.shape))
    show_distribution(y_extended, 'Distribution of Signs - Extended Training Set', 'output/extended_training_data_distribution.png')

    X_extended = np.concatenate((X, X_extended))
    y_extended = np.concatenate((y, y_extended))
    print('Merged with original training set. X_extended.shape: {}, y_extended.shape: {}'.format(X_extended.shape, y_extended.shape))
    show_distribution(y_extended, 'Distribution of Signs - Merged Training Set', 'output/merged_training_data_distribution.png')
    return (X_extended, y_extended)

#save_and_show_augmented_image()


In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
import cv2

def show_rgb_gray_images(rgb_image, gray_image):
    x_len = rgb_image.shape[0]
    plt.figure(figsize=(6, 6))
    plt.subplot(121)
    plt.xticks(np.arange(0, x_len, 10))
    plt.imshow(rgb_image)
    plt.subplot(122)
    plt.xticks(np.arange(0, x_len, 10))
    plt.imshow(gray_image, cmap='gray')
    plt.savefig('output/rgb_gray_images.png', bbox_inches='tight')

def convert_gray(X):
    gray_images = []
    for image in X:
        gray_images.append(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY))
    gray_image_array = np.stack(gray_images)
    return np.expand_dims(gray_image_array, axis=3)
    
def convert_gray_2(X_data):
    gray_X = None
    rgb_images = tf.placeholder(tf.int32, (None, 32, 32, 3))
    convert_color = tf.image.rgb_to_grayscale(rgb_images)
    with tf.Session() as sess:
        gray_X = sess.run(convert_color, feed_dict={rgb_images: X_data})
    return gray_X

def preprocess_image(img, convert_to_grayscale = False):
    new_img = None
    if convert_to_grayscale:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        gray_img = np.expand_dims(gray_img, axis = 2)
        new_img = (gray_img.astype(np.float32) - 128) / 128
    else:
        new_img = (img.astype(np.float32) - 128) / 128
    return new_img

def normalize(X): 
    return (X.astype(np.float32) - 128) / 128

def preprocess(X, convert_to_grayscale): 
    new_X = None
    if convert_to_grayscale:
        gray_X = convert_gray(X)
        new_X = (gray_X.astype(np.float32) - 128) / 128
    else:
        new_X = (X.astype(np.float32) - 128) / 128
        
    return new_X

convert_to_grayscale = True
channel_number = 3

# Convert to grayscale if needed
X_train_data = None
if convert_to_grayscale:
    channel_number = 1
    X_train_data = convert_gray(X_train)
    # Randomly pick and show RGB / Gray images for comparison
    index = np.random.randint(0, n_train)
    show_rgb_gray_images(X_train[index].squeeze(), X_train_data[index].squeeze())
else:
    X_train_data = X_train

X_extended, y_extended = run_augmentation(X_train_data, y_train)
X_train_normal = preprocess(X_extended, False)
X_valid_normal = preprocess(X_valid, convert_to_grayscale)
X_test_normal = preprocess(X_test, convert_to_grayscale)


### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
def LeNet(x, n_classes, channel_number, keep_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # Layer 1: Convolutional. Input = 32x32x"channel_number", Output = 28x28x6.
    C1_W = tf.Variable(tf.truncated_normal((5, 5, channel_number, 6), mean = mu, stddev = sigma))
    C1_b = tf.Variable(tf.zeros(6))
    C1 = tf.nn.conv2d(x, C1_W, strides = [1, 1, 1, 1], padding = 'VALID') + C1_b
    # Activation.
    C1 = tf.nn.relu(C1)

    # Pooling. Input = 28x28x6. Output = 14x14x6.
    S2 = tf.nn.max_pool(C1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Layer 2: Convolutional. Output = 10x10x16.
    C3_W = tf.Variable(tf.truncated_normal((5, 5, 6, 16), mean = mu, stddev = sigma))
    C3_b = tf.Variable(tf.zeros(16))
    C3 = tf.nn.conv2d(S2, C3_W, strides = [1, 1, 1, 1], padding = 'VALID') + C3_b

    # Activation.
    C3 = tf.nn.relu(C3)
    
    # Pooling. Input = 10x10x16. Output = 5x5x16.
    S4 = tf.nn.max_pool(C3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Flatten. Input = 5x5x16. Output = 400.
    Flat_4 = flatten(S4)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    FC5_W = tf.Variable(tf.truncated_normal((400, 120), mean = mu, stddev = sigma))
    FC5_b = tf.Variable(tf.zeros(120))
    FC_5 = tf.add(tf.matmul(Flat_4, FC5_W), FC5_b)

    # Activation.
    FC_5 = tf.nn.relu(FC_5)
    # First Dropout
    FC_5 = tf.nn.dropout(FC_5, keep_prob)

    # Layer 4: Fully Connected. Input = 120. Output = 84.
    FC6_W = tf.Variable(tf.truncated_normal((120, 84), mean = mu, stddev = sigma))
    FC6_b = tf.Variable(tf.zeros(84))
    FC_6 = tf.add(tf.matmul(FC_5, FC6_W), FC6_b)
    
    # Activation.
    FC_6 = tf.nn.relu(FC_6)
    # Second Dropout
    FC_6 = tf.nn.dropout(FC_6, keep_prob)
    
    # Layer 5: Fully Connected. Input = 84. Output = n_classes.
    logits_W = tf.Variable(tf.truncated_normal((84, n_classes), mean = mu, stddev = sigma))
    logits_b = tf.Variable(tf.zeros(n_classes))
    logits = tf.add(tf.matmul(FC_6, logits_W), logits_b)
    
    feature_activations = [C1, S2, C3, S4]
    return (logits_W, logits, feature_activations)


### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
import tensorflow as tf
### Define your model here.
x = tf.placeholder(tf.float32, (None, 32, 32, channel_number))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)
keep_prob = tf.placeholder(tf.float32) # probability to keep units

rate = 0.001
EPOCHS = 20
BATCH_SIZE = 128
beta = 0.01

(logits_W, logits, feature_activations) = LeNet(x, n_classes, channel_number, keep_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
#regularizer = tf.nn.l2_loss(logits_W)
#loss_operation = tf.reduce_mean(loss_operation + beta * regularizer)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
### Calculate and report the accuracy on the training and validation set.
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    total_loss = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        end = offset + BATCH_SIZE
        batch_x, batch_y = X_data[offset:end], y_data[offset:end]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
        loss = sess.run(loss_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_loss += (loss * len(batch_x))
    return (total_accuracy / num_examples, total_loss / num_examples)


In [ ]:
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
from sklearn.utils import shuffle
import time

def plotAndSaveAccuracies(x, training_accuracies, validation_accuracies):
    plt.figure()
    plt.plot(x, training_accuracies, 'b-')
    plt.plot(x, validation_accuracies, 'r-')
    labels = ['Training Accuracy', 'Validation Accuracy']
    #plt.legend(labels, loc = 'center left', bbox_to_anchor=(1, 0.5))
    plt.legend(labels, loc = 'lower right')
    plt.xticks(x_epochs)
    plt.yticks(np.arange(0.5, 1.05, 0.05))
    plt.grid()
    filename = 'output/accuracies_' + str(channel_number) + '.png'
    plt.savefig(filename)
    plt.show()

def plotAndSaveLosses(x, training_losses, validation_losses):
    plt.figure()
    plt.plot(x_epochs, training_losses, 'b--')
    plt.plot(x_epochs, validation_losses, 'r--')
    labels = ['Training Losses', 'Validation Losses']
    plt.legend(labels, loc = 'upper right')
    plt.xticks(x_epochs)
    plt.grid()
    filename = 'output/losses_' + str(channel_number) + '.png'
    plt.savefig(filename)
    plt.show()
    
saver = tf.train.Saver()
start_time = time.time()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("Training...")
    print()
    training_accuracies = []
    training_losses = []
    validation_accuracies = []
    validation_losses = []
    X_train_set = X_train_normal
    y_train_set = y_extended
    n_train_set = X_train_set.shape[0]
    for i in range(EPOCHS):
        #X_train_normal, y_train = shuffle(X_train_normal, y_train)
        X_train_set, y_train_set = shuffle(X_train_set, y_train_set)
        for offset in range(0, n_train_set, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_set[offset:end], y_train_set[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})
        
        training_accuracy, training_loss = evaluate(X_train_set, y_train_set)
        validation_accuracy, validation_loss = evaluate(X_valid_normal, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        training_accuracies.append(training_accuracy)
        training_losses.append(training_loss)
        validation_accuracies.append([validation_accuracy])
        validation_losses.append([validation_loss])
        print()

    x_epochs = np.arange(1, EPOCHS + 1, 1.0)
    plotAndSaveAccuracies(x_epochs, training_accuracies, validation_accuracies)
    plotAndSaveLosses(x_epochs, training_losses, validation_losses)

    print('Time used for training model: {:.1f}'.format(time.time() - start_time))
    saver.save(sess, './lenet')
    print("Model saved")

In [ ]:
# Evaluate on test set. Shall run only once
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    test_accuracy, test_loss = evaluate(X_test_normal, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))
    print("Test Loss = {:.3f}".format(test_loss))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
import os
import matplotlib.image as mpimg

image_dir = "downloaded/"
image_files = ['pedestrians.jpg', 'bumpy_road.jpg', 'speed_limit_30.jpg', 
               'stop_sign.jpg', 'turn_left.jpg', 'turn_right.jpg' ]
plt.figure(figsize=(3, 2))

index = 1
print('Downloaded traffic signs:')
for file in image_files:
    img = mpimg.imread(image_dir + '/' + file)
    plt.subplot(2, 3, index)
    plt.imshow(img)
    index += 1


### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
image_list = []
for file in image_files:
    img = mpimg.imread(image_dir + '/' + file)
    image_list.append(preprocess_image(img, convert_to_grayscale))
X_downloaded = np.stack(image_list)
y_downloaded = [27, 22, 1, 14, 34, 33]

saver = tf.train.Saver()
predict_sign_operation = tf.argmax(logits, 1)
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    predicted_signs = sess.run(predict_sign_operation, feed_dict={x: X_downloaded, keep_prob: 1.0})
    print('Prediction:')
    index = 1
    for sign in predicted_signs:
        print('Image #{}: {} ({})'.format(index, signnames[str(sign)], sign))
        index += 1

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

correct = np.sum(np.equal(predicted_signs, y_downloaded))
print('Prediction Accuracy: {:.3f} ({}/{})'.format(correct / len(predicted_signs), correct, len(predicted_signs)))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.
logits_softmax_operation = tf.nn.softmax(logits)
def logits_prediction(X_data):
    with tf.Session() as sess:
        saver.restore(sess, tf.train.latest_checkpoint('.'))
        predictions = sess.run(logits_softmax_operation, feed_dict={x: X_data, keep_prob: 1.0})
        return predictions

predictions = logits_prediction(X_downloaded)
i_image = 1
for prediction in predictions:
    top_sign_ids = prediction.argsort()[-5:][::-1]
    print('Prediction of image #{}:'.format(i_image))
    for sign_id in top_sign_ids:
        print('Probablity of {} ({}): {}'.format(signnames[str(sign_id)], sign_id, prediction[sign_id]))
    print()
    i_image += 1

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

layer_names = {1: 'Convolutional Layer 1', 2: 'Max Pooling Layer 1', 
               3: 'Convolutional Layer 2', 4: 'Max Pooling Layer 2'}

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    with tf.Session() as sess:
        saver.restore(sess, tf.train.latest_checkpoint('.'))
        activation = sess.run(tf_activation, feed_dict={x: image_input, keep_prob: 1.0})
#   activation = tf_activation.eval(session=sess,feed_dict={x : image_input, keep_prob: 1.0})
        featuremaps = activation.shape[3]
        plt.figure(plt_num, figsize=(15, 15))
        plt.suptitle('Figure of {}'.format(layer_names[plt_num]))

        for featuremap in range(featuremaps):
            plt.subplot(6,8, featuremap + 1) # sets the number of feature maps to show on each row and column
            plt.title('FeatureMap ' + str(featuremap + 1)) # displays the feature map number
            if activation_min != -1 & activation_max != -1:
                plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
            elif activation_max != -1:
                plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
            elif activation_min !=-1:
                plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
            else:
                plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")
        plt.subplots_adjust(top=0.95)
        plt.savefig('output/featuremap_pltnum_{}.png'.format(plt_num), bbox_inches='tight')
        return activation

tf_result = []
image_input = X_downloaded[None, 2]
plt_num = 1
for tf_activation in feature_activations:
    tf_result.append(outputFeatureMap(image_input, tf_activation, plt_num = plt_num))
    plt_num += 1